In [1]:
import numpy as np

# Chapter 4. word2vec 속도 개선

## \# 4.1 word2vec 개선 1 : Embedding Layer

- 앞서 구현했던 `MatMul` layer의 방식으로는 너무 많은 계산을 하게 됨.


- 사실 one-hot vector인 input 벡터와 `W_in` 가중치의 내적은 one-hot vector의 특성 상 그냥 `W_in` 가중치의 특정 행을 추출하는 것


- 따라서 내적 곱을 하는 layer를 이용하지 않고 단순히 slicing으로 `W_in` 의 특정 행을 추출하는 `Embedding Layer`를 만들어 사용

In [2]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        
        # dW의 모든 값을 0으로 덮어씌우기
        dW[...] = 0
        
        # idx에 해당하는 행에 dout값을 모두 더해주기
        np.add.at(dW, self.idx, dout)
        return None

#### 예제 text

In [3]:
text = "Thomas Jefferson once said I'm a great believer in luck, and I find the harder I work, \
the more I have of it. What, though, is luck? Webster's dictionary suggests that luck is the \
events or circumstances that operate for or against an individual. In truth, luck has nothing \
to do with something operating for or against you. Luck is not a matter of chance. \
It is a matter of being open to new experiences, perseverance, hard work, and positive thinking. \
When seventeen year old Steven Spielberg spent some time with his cousin in the summer of 1965, \
they toured Universal pictures. The tram stopped at none of the sound stages. \
Spielberg snuck off on a bathroom break to watch a bit of the real action. \
When he encountered an unfamiliar face who demanded to know what he was doing, \
he told him his story. The man turned out to be the head of the editorial department. \
Spielberg got a pass to the lot for the very next day and showed a very impressed \
Chuck Silvers four of his eight millimeter films. This was the foot in the door Spielberg \
needed to start squatting on the lot, a decision that led to his first contract with Universal Studios. \
Studies have shown that lucky people tend to be far more open to new experiences. Those who are \
unlucky are creatures of habit, never varying from one day to the next. If you want to be lucky, \
add some variety to your life. Meet new people, go to new places, and increase the possibility of \
those chance opportunities the lucky people always seem to run into."

#### preprocess 함수로 말뭉치 corpus 생성

In [4]:
from common.util import preprocess

In [5]:
corpus, word_to_id, id_to_word = preprocess(text)
corpus[:50]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,  4, 12, 13, 14,  4,
       15, 13, 16,  4, 17, 18, 19, 20, 21, 22, 23, 10, 24, 25, 26, 27, 28,
       29, 10, 23, 13, 30, 31, 32, 29, 33, 34, 31, 35, 36, 37, 20,  9])

#### unique한 단어는 165개

In [6]:
len(word_to_id)

165

#### 임시방편으로 W는 165개의 단어별로 3개의 노드를 갖는 (165 x 3) shape로 생성

In [7]:
W = np.arange(165*3).reshape(165, 3)
W[:5]

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

#### 추출하고자 하는 단어 선택

In [8]:
context = ['very', 'day']
context_idx = [word_to_id[word] for word in context]
context_idx

[112, 114]

In [9]:
target = ['next']
target_idx = [word_to_id[word] for word in target]
target_idx

[113]

#### `Embedding` class의 forward 메서드를 이용해서 idx에 해당하는 행을 바로 추출 가능

In [10]:
embed = Embedding(W)

context_W = embed.forward(context_idx)
target_W = embed.forward(target_idx)

print('contexts: ', context_W, '\n\n', 'target: ', target_W, sep='')

contexts: [[336 337 338]
 [342 343 344]]

target: [[339 340 341]]


## \# 4.2.4 다중분류에서 이중분류로 : `EmbeddingDot`

- `W_out` 쪽에서의 연산 또한 계산량이 너무 크다는 문제가 있다.


- 'target 단어가 'say'입니까?' 라고 묻는 기존의 다중 분류 방식 `multi-class classification`에서,    
'target 단어가 'say'가 맞습니까?' 라고 묻는 이진 분류 방식 `binary class classification`으로 변경



### 연산 과정
- target 단어인 say를 idx 형태로 입력받으면 Embedding class로 forward 시켜서 target_W 를 출력


- 출력된 target_W 의 열벡터를 입력받는 h벡터 (입력으로 받은 context들로부터 생성된 벡터) 와 dot 연산을 통해 out 값 (scalar값)을 출력

In [11]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
    
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        # dot 연산에 대한 backward
        # dout의 값에 각각 서로를 바꿔서 곱해준 값이 gradient값
        dtarget_W = dout * h
        dh = dout * target_W
        # embed 계층 쪽에도 backward 수행
        self.embed.backward(dtarget_W)
        return dh

#### `h` : context 들의 context_W를 합쳐서 평균낸 벡터

In [12]:
h0 = context_W[0] 
h1 = context_W[1]
h = (h0 + h1) / 2
h

array([339., 340., 341.])

#### EmbeddingDot 의 forward

In [13]:
embed_dot = EmbeddingDot(W)

out = embed_dot.forward(h, target_idx)
out

array([346802.])

#### EmbeddingDot 의 backward

In [14]:
dout = np.array([1])
dh = embed_dot.backward(dout)
dh

array([[339, 340, 341]])

## \# 4.2.5 Negative Sampling의 sampling 기법 : `UnigramSampler`

- 다중분류에서 이진분류로 옮겨오면서 '틀린' 경우에 대한 학습을 모든 어휘에 대해 하지 않고 특정 단어들을 샘플링해서 일부분만 학습하는 방법을 취한다.


- 예를 들어, context가 `'you'`, `'goodbye'`일 때 정답은 `'say'`인 경우를 생각해보자.


- 모델은 'say'를 `label 1`로 **한 번** 학습하고, 그 외의 틀린 단어들을 `label 0`으로 **여러 번** 학습한다. 


- 이 때 틀린 단어들 (negative samples)에 대해서는 특정 개수를 정해서 **그 횟수만큼만** 학습한다.    
(연산량을 줄이기 위해 꼭 필요한 만큼만 학습)


- 이 때 sampling 에 쓰이는 작은 트릭은, 
**많이 쓰이는 단어들에 대해서는 sampling될 확률을 좀 더 크게 주고, 
잘 안쓰이는 단어에 대해서는 확률을 좀 작게 주는 것.**


- 이로써 더 많이 쓰이는 단어들에 대해 더 많이 학습하고 적게 쓰이는 단어들은 적게 학습하는 효과를 기대할 수 있다.    
(별로 안쓰이는 단어를 많이 학습할 필요는 없다는 말)

### negative sampling을 위한 `UnigramSampler`

In [15]:
import collections

In [16]:
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = len(set(corpus))
        self.word_p = np.zeros(self.vocab_size)
        
        # 각 단어가 몇 번씩 나오는지 저장
        counts = collections.Counter(corpus)
        
        # 각 단어의 출현횟수를 확률로 변환하여 word_p에 저장
        self.word_p = [counts[i] for i in range(self.vocab_size)]
        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)
        
    def get_negative_sample(self, target):
        batch_size = target.shape[0]
        
        negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

        for i in range(batch_size):
            p = self.word_p.copy()

            # target은 뽑히지 않도록 하기 위해 확률을 0으로 설정
            target_idx = target[i]
            p[target_idx] = 0

            p /= p.sum()
            negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, \
                                                     replace=False, p=p)
            
        return negative_sample

#### 각 단어에 대한 확률값 확인

In [17]:
uni_sampler = UnigramSampler(corpus, power=0.75, sample_size=5)

print(len(uni_sampler.word_p))
uni_sampler.word_p[:10]

165


array([0.00421884, 0.00421884, 0.00421884, 0.00421884, 0.01193269,
       0.00421884, 0.02006832, 0.00421884, 0.00421884, 0.01193269])

#### 전체 합 1임을 확인

In [18]:
sum(uni_sampler.word_p)

1.0000000000000009

#### negative_sample 해보기

In [19]:
target = np.array([5, 10, 3, 2, 45])

negative_sample = uni_sampler.get_negative_sample(target)
negative_sample

array([[159,   2, 128, 158,  50],
       [108,  52,  44, 133, 138],
       [ 11, 123,  28,  23,  61],
       [ 11, 144, 155,   6,  43],
       [ 54, 122, 102, 114,  41]], dtype=int32)

- 확률이 높은 6이 두 번 나왔고, target값은 제회하고 sampling 된 것을 확인할 수 있다.


- 6은 무슨 단어일까?

In [20]:
id_to_word[6]

'a'

- 'a'로, 많이 나오는 단어가 맞다!

## \# 4.2.7 Negative Sampling 구현

- 먼저 `SigmoidWithLoss` layer 를 구현한다.

In [21]:
from common.layer import Sigmoid
from common.function import cross_entropy_error

In [22]:
class SigmoidWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = 1 / (1 + np.exp(-x))
        
        self.loss = cross_entropy_error(np.c_[1 - self.y, self.y], self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        
        dx = (self.y - sefl.t) * dout / batch_size
        return dx

#### `NegativeSamplingLoss` 구현

In [23]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        
        # negative sample size에 정답 sample을 위해 1을 더해줌
        self.loss_layers = [sigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
        
        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)
        
        # Positive sample
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)
        
        # Negative sample
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1+i].forward(h, negative_target)
            loss += self.loss_layers[1+i].forward(score, negative_label)
            
        return loss
    
    
    def backward(self, dout=1):
        dh = 0
        for loss_layer, embed_dot_layer in zip(self.loss_layers, self.embed_dot_layers):
            dscore = loss_layer.backward(dout)
            dh += embed_dot_layer.backward(dscore)
            
        return dh